In [ ]:
from pipeline.Preprocessing import *
from pipeline.featureExtraction import *
from pipeline.Colourspaces import *
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
import joblib

In [ ]:
## the following step to test an image on a pre-trained model

#1. Read 
image= cv2.imread("BoWFireDataset/dataset/img/fire/fire007.png")

In [ ]:
#2 resize and bilateral filter (preprocessing step)
img_resized = resize_with_padding(image=image)
blured_img = apply_bilateral_filter(img=img_resized)

height, width, _ = blured_img.shape # to use later after prediction to convert array of pixels to 2D

In [ ]:
#3 
blured_img= cv2.cvtColor(blured_img, cv2.COLOR_BGR2RGB) # this step is critical cuz the filters assume input is in RGB not BGR(default)
# 3.1Extract pixel color features : selected colors are ['V', 'b', 'Cr', 'Cb']
color_feats = extract_top_color_features(blured_img)
# 3.2 Extract gabor features : selected gabor resonses are ['g2', 'g6', 'g10', 'g14']
gabor_features= extract_top_gabor_features(blured_img)
combined_feats = np.hstack((color_feats, gabor_features))  # (n_pixels, 8)


In [ ]:

#4 need to scale the input: I should have passed the same scaler I used
#  for training or now!! I add this later
scaler = joblib.load("tuned_models/scaler.pkl")
features_scaled = scaler.transform(combined_feats)

In [ ]:
#5. Load the saved model and predict
model = joblib.load('tuned_models/best_logistic_model.pkl')
y_pred = model.predict(features_scaled) #1D array of pixels


In [ ]:
#6 reshape from array to 2D
mask = y_pred.reshape((height, width))

plt.imshow(mask, cmap='hot')
plt.title("Fire Prediction Mask")
plt.axis('off')
plt.show()

rbgImg = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
plt.imshow(rbgImg)
plt.show()